In [1]:
import xml.etree.ElementTree as ET

import pandas as pd
from datetime import datetime



In [2]:
ugly = '/home/tony/opt/dvr1/5_tv_guide/xmltv.xml'


In [3]:
def tv_df_make(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    # create channel lookup table
    chans_list = []
    for chan_elem in root.findall('.//channel'):
        chan_dict = {}
        chan_dict['id'] = chan_elem.get('id')
        i=0
        for child_elem in chan_elem.findall('display-name'):
            #print (child_elem.text)
            tag=child_elem.tag+str(i)
            chan_dict[tag] = child_elem.text
            i += 1
        chans_list.append(chan_dict)
    CHAN_DF = pd.DataFrame(chans_list)
    CHAN_DICT = CHAN_DF.to_dict('records')
    CHAN_LOOKUP={}
    for rec in CHAN_DICT:
        CHAN_LOOKUP[rec['id']] = rec['display-name1']
    
    
    # Create a list to store dictionaries representing each book
    progr_list = []
    # Iterate through each <book> element
    for progr_elem in root.findall('.//programme'):
        progr_dict = {}
        progr_dict['start'] = progr_elem.get('start')
        progr_dict['stop'] = progr_elem.get('stop')
        progr_dict['channel'] = progr_elem.get('channel')
        #progr_dict['title'] = progr_elem.get('title')
    
    
        i=0
        for child_elem in progr_elem.findall('title'):
            # print (child_elem.text)
            tag=child_elem.tag
            progr_dict[tag] = child_elem.text
            i += 1
            
        for child_elem in progr_elem.findall('sub-title'):
            # print (child_elem.text)
            tag=child_elem.tag
            progr_dict[tag] = child_elem.text
            
        for child_elem in progr_elem.findall('length'):
            # print (child_elem.text)
            tag=child_elem.tag
            progr_dict[tag] = child_elem.text
    
    
        progr_list.append(progr_dict)
    pdf = pd.DataFrame(progr_list)
    pdf['vchan'] = pdf['channel'].map(CHAN_LOOKUP)
    pdf['start'] = pd.to_datetime(pdf['start'])
    pdf['stop'] = pd.to_datetime(pdf['stop'])
    pdf['vchan'] = pdf['vchan'].astype(float) 


    now = datetime.now()
    local_timezone = now.astimezone().tzinfo

    print("Current timezone:", local_timezone)
    
    pdf['start'] = pdf['start'].dt.tz_convert(local_timezone)
    pdf['stop'] = pdf['stop'].dt.tz_convert(local_timezone)
    #pdf['start'] = pdf['start'].dt.tz_localize(local_timezone) 
    #pdf['stop'] = pdf['stop'].dt.tz_localize(local_timezone) 

    pdf['sub-title'] = pdf['sub-title'].fillna('ONE')
    pdf = pdf.sort_values(by='vchan')
    
    pdf = pdf[['vchan', 'start', 'stop', 'title', 'sub-title', 'length']]


    # Display the DataFrame
    print(pdf.head())
    return pdf
    

In [4]:
xml_file = ugly
df = tv_df_make(xml_file)

Current timezone: CST
       vchan                     start                      stop  \
4957     2.1 2025-01-21 03:00:00-06:00 2025-01-21 04:00:00-06:00   
4958     2.1 2025-01-21 04:00:00-06:00 2025-01-21 05:00:00-06:00   
4959     2.1 2025-01-21 05:00:00-06:00 2025-01-21 05:30:00-06:00   
4960     2.1 2025-01-21 05:30:00-06:00 2025-01-21 06:00:00-06:00   
11451    2.1 2025-01-24 06:00:00-06:00 2025-01-24 06:30:00-06:00   

                                              title  \
4957   All Creatures Great and Small on Masterpiece   
4958                                   Vienna Blood   
4959                                   NHK Newsline   
4960               Classical Stretch: By Essentrics   
11451                                      BBC News   

                                               sub-title length  
4957                                    Holding the Baby     60  
4958                                    The Enemy Within     60  
4959                                    

In [5]:
df.columns

Index(['vchan', 'start', 'stop', 'title', 'sub-title', 'length'], dtype='object')

In [6]:
type(df)

pandas.core.frame.DataFrame

In [7]:
df.dtypes

vchan                    float64
start        datetime64[ns, CST]
stop         datetime64[ns, CST]
title                     object
sub-title                 object
length                    object
dtype: object

In [8]:
# Get current time

now = datetime.now()
local_timezone = now.astimezone().tzinfo


now_aware = datetime.now(local_timezone)
now = now_aware
#now = datetime.now()

# Filter DataFrame to find rows with 'now' between 'start' and 'stop'
filtered_df = df[(df['start'] <= now) & (now <= df['stop'])]

filtered_df.head()


,vchan,start,stop,title,sub-title,length
2274,2.1,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,Sister Boniface Mysteries,A Tight Squeeze,60
2277,2.2,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The Age of Nature,Understanding,60
2281,2.3,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Cook's Country,Biscuits and Chicken,30
2286,2.4,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Arthur,Muffy Misses Out; Arthur Takes a Stand,30
2509,2.6,2025-01-19 18:00:00-06:00,2025-01-19 22:00:00-06:00,Classical 24,ONE,240


In [9]:
df.columns.values

array(['vchan', 'start', 'stop', 'title', 'sub-title', 'length'],
      dtype=object)

In [10]:
df = filtered_df

In [11]:
# # Sort the DataFrame by 'vchan'
# df_sorted = df.sort_values(by='vchan')

# # Select only the 'vchan', 'start', and 'title' columns
# df_pruned = df_sorted[['vchan', 'start', 'title']]

# # Print the resulting DataFrame
# print(df_pruned) 

In [12]:
#df_pruned.head(30)

df.head(20)

,vchan,start,stop,title,sub-title,length
2274,2.10,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,Sister Boniface Mysteries,A Tight Squeeze,60
2277,2.20,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The Age of Nature,Understanding,60
2281,2.30,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Cook's Country,Biscuits and Chicken,30
2286,2.40,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Arthur,Muffy Misses Out; Arthur Takes a Stand,30
2509,2.60,2025-01-19 18:00:00-06:00,2025-01-19 22:00:00-06:00,Classical 24,ONE,240
2290,2.60,2025-01-19 18:00:00-06:00,2025-01-19 22:00:00-06:00,Classical 24,ONE,240
2291,4.10,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The Hunting Party,Richard Harris,60
2294,4.20,2025-01-19 18:00:00-06:00,2025-01-19 20:00:00-06:00,Bring It On,Movie: 2000,120
2298,4.30,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,M*A*S*H,Out of Gas,30
2303,4.40,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The FBI Files,Dangerous Cause,60


In [14]:
df_no_duplicates = df.drop_duplicates() 

In [15]:
df_no_duplicates.head(40)

,vchan,start,stop,title,sub-title,length
2274,2.10,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,Sister Boniface Mysteries,A Tight Squeeze,60
2277,2.20,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The Age of Nature,Understanding,60
2281,2.30,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Cook's Country,Biscuits and Chicken,30
2286,2.40,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Arthur,Muffy Misses Out; Arthur Takes a Stand,30
2509,2.60,2025-01-19 18:00:00-06:00,2025-01-19 22:00:00-06:00,Classical 24,ONE,240
2291,4.10,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The Hunting Party,Richard Harris,60
2294,4.20,2025-01-19 18:00:00-06:00,2025-01-19 20:00:00-06:00,Bring It On,Movie: 2000,120
2298,4.30,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,M*A*S*H,Out of Gas,30
2303,4.40,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The FBI Files,Dangerous Cause,60
2306,4.50,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,NCIS,Death From Above,60


In [16]:
#ndf = df['vchan'].isin(['11.1', '11.2'])

In [17]:
sdf = df_no_duplicates
sdf['vchan'] = sdf['vchan'].astype(str)
#filtered_df = sdf[sdf['vchan'].contains(['11.', '13.'])]
filtered_df = sdf[sdf['vchan'].str.startswith('11.')]

/tmp/ipykernel_603073/2066567371.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['vchan'] = sdf['vchan'].astype(str)


In [18]:
filtered_df

,vchan,start,stop,title,sub-title,length
2118,11.1,2025-01-19 17:30:00-06:00,2025-01-19 21:00:00-06:00,NFL Football,Baltimore Ravens at Buffalo Bills,210
2356,11.2,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Fishing Friendsy,ONE,30
2359,11.3,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,NCIS,Death From Above,60
2362,11.4,2025-01-19 18:00:00-06:00,2025-01-19 20:00:00-06:00,Bring It On,Movie: 2000,120


In [19]:
filtered_df.dtypes

vchan                     object
start        datetime64[ns, CST]
stop         datetime64[ns, CST]
title                     object
sub-title                 object
length                    object
dtype: object

In [20]:
sdf

,vchan,start,stop,title,sub-title,length
2274,2.1,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,Sister Boniface Mysteries,A Tight Squeeze,60
2277,2.2,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The Age of Nature,Understanding,60
2281,2.3,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Cook's Country,Biscuits and Chicken,30
2286,2.4,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Arthur,Muffy Misses Out; Arthur Takes a Stand,30
2509,2.6,2025-01-19 18:00:00-06:00,2025-01-19 22:00:00-06:00,Classical 24,ONE,240
...,...,...,...,...,...,...
2475,46.3,2025-01-19 16:30:00-06:00,2025-01-19 19:00:00-06:00,Movie,ONE,150
2477,46.4,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,Monk,Mr. Monk Makes a Friend,60
2482,46.5,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The FBI Files,Dangerous Cause,60
2485,46.6,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,Hawaii Five-0,Kalele,60


In [21]:
import pandas as pd

# Sample DataFrame
data = {'vchan': ['ABC1', 'HBO2', 'CNN1', 'FOX3', '11.1', '11.2', '13.5', 'ABC4']}
df = pd.DataFrame(data)

# Convert 'vchan' to string (if not already)
df['vchan'] = df['vchan'].astype(str)

# Filter DataFrame using str.startswith()
filtered_df = df[df['vchan'].str.startswith('11.') | df['vchan'].str.startswith('13.')]

print(filtered_df)

  vchan
4  11.1
5  11.2
6  13.5


In [22]:
filtered_df.dtypes

vchan    object
dtype: object

In [23]:
fav_chans = [11,13,46]

In [24]:
dfs=[]
df = sdf
for chan in fav_chans:
    vmatch = f'{chan}.'
    print(vmatch)
    fdf = df[df['vchan'].str.startswith(vmatch)]
    #print(fdf)
    dfs.append(fdf)

11.
13.
46.


In [25]:
bdf = pd.concat(dfs)

In [26]:
bdf.head(20)

,vchan,start,stop,title,sub-title,length
2118,11.1,2025-01-19 17:30:00-06:00,2025-01-19 21:00:00-06:00,NFL Football,Baltimore Ravens at Buffalo Bills,210
2356,11.2,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Fishing Friendsy,ONE,30
2359,11.3,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,NCIS,Death From Above,60
2362,11.4,2025-01-19 18:00:00-06:00,2025-01-19 20:00:00-06:00,Bring It On,Movie: 2000,120
2365,13.1,2025-01-19 18:00:00-06:00,2025-01-19 19:01:00-06:00,America's Funniest Home Videos,ONE,61
2368,13.2,2025-01-19 18:00:00-06:00,2025-01-19 21:00:00-06:00,The Outlaw Josey Wales,Movie: 1976,180
2370,13.3,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,M*A*S*H,Out of Gas,30
2375,13.4,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,Homicide Hunter: Lt. Joe Kenda,Moebus and Watson,60
2378,13.5,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The Closer,Elysian Fields,60
2382,13.6,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,Rocky & Bullwinkle,"Box Top Robbery, Parts 1 & 2",30


In [27]:
fav_chans = [4,7,23,36, 44]

In [28]:
dfs=[]
df = sdf
for chan in fav_chans:
    vmatch = f'{chan}.'
    print(vmatch)
    fdf = df[df['vchan'].str.contains(vmatch)]
    #print(fdf)
    dfs.append(fdf)

4.
7.
23.
36.
44.


In [29]:
bdf = pd.concat(dfs)
bdf.head(38)

,vchan,start,stop,title,sub-title,length
2291,4.1,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The Hunting Party,Richard Harris,60
2294,4.2,2025-01-19 18:00:00-06:00,2025-01-19 20:00:00-06:00,Bring It On,Movie: 2000,120
2298,4.3,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,M*A*S*H,Out of Gas,30
2303,4.4,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,The FBI Files,Dangerous Cause,60
2306,4.5,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,NCIS,Death From Above,60
2385,14.1,2025-01-19 18:30:00-06:00,2025-01-19 19:00:00-06:00,The Jamie Foxx Show,"I Do, I Didn't",30
2390,14.2,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,Without a Trace,Hang on to Me,60
2393,14.3,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,Grimm,Season of the Hexenbiest,60
2396,14.4,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,Only in America With Larry the Cable Guy,American Guns,60
2454,44.0,2025-01-19 18:00:00-06:00,2025-01-19 22:00:00-06:00,To Be Announced,ONE,240


In [30]:
fdf = df[df['title'].str.contains('48')]


In [31]:
fdf

,vchan,start,stop,title,sub-title,length
2333,7.7,2025-01-19 18:00:00-06:00,2025-01-19 19:00:00-06:00,48 Hours,Every Picture Tells a Story,60
